### web scraping (só pelo o id)

In [5]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW
import requests, sys, json
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np
from tqdm import tqdm
from Bio import SearchIO
from Bio.SwissProt import KeyWList
from Bio import SwissProt
from Bio import ExPASy
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
from Bio import Phylo

In [10]:
import time
def url_get(i):
    url_list= [ ]
    for id in range(243,(243+i)):
        url = "https://www.ncbi.nlm.nih.gov/nuccore/NG_009{}".format( id )
        url_list.append(url)
    return url_list
url_get(4)

['https://www.ncbi.nlm.nih.gov/nuccore/NG_009243',
 'https://www.ncbi.nlm.nih.gov/nuccore/NG_009244',
 'https://www.ncbi.nlm.nih.gov/nuccore/NG_009245',
 'https://www.ncbi.nlm.nih.gov/nuccore/NG_009246']

In [12]:
import requests

content = []
for url in url_get(4):
    r = requests.get(url)
    content.append(r.content)
#print(content)

In [14]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

    id = ""
    url = ""
    for line in lines:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id = line.attrs['content']
    if id:
        url = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id)

    r2 = requests.get( url )
    r3= r2.content.decode('utf-8')
    #r3= str(r2.content)
    #print (r3, '\n')

<meta content="325910839" name="ncbi_uidlist"/>
ncbi_uidlist
325910839
<meta content="1543376889" name="ncbi_uidlist"/>
ncbi_uidlist
1543376889
<meta content="219521908" name="ncbi_uidlist"/>
ncbi_uidlist
219521908
<meta content="219521909" name="ncbi_uidlist"/>
ncbi_uidlist
219521909


In [ ]:
#import re
#existe = re.findall(r"ORGANISM\s+.*?(?=bp)", r3, re.DOTALL)
#if existe:
    #for definition in existe:
        #m = re.match( r"ORGANISM\s+(.+)", definition, re.DOTALL )
        #print( re.sub(r'\s+', ' ', m.group(1) ) )

### web scraping (por condição)

In [29]:
import time
query= input('escolha o que quer pesquisar: ')
def url_get(i):
    url_list= [ ]
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format( query )
    url_list.append(url)
    return url_list
url_get(1)

import requests, sys, json
content = []
for url in url_get(1):
    r = requests.get(url)
    content.append(r.content)
#print(content)

import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(c, 'html.parser')
    a= soup.get_text()
    #print (a)

import re
existe = re.findall(r"ID:\s+\d*(?=\D)", a, re.DOTALL)

c= ', '.join(existe)
h= c.replace('ID: ','')
IDS= h.split(', ')

n_genes= IDS[0:int(input('escolha o nº de genes que quer obter (máximo 20): '))]
#print(n_genes)

if n_genes == ['']:
    print ("Não encontramos resultado para a sua pesquisa")
else:
    print (n_genes)

    


escolha o que quer pesquisar: irs1
escolha o nº de genes que quer obter (máximo 20): 10
['3667', '3667', '16367', '25467', '559281', '3077574', '100512686', '538598', '459985', '100607873']


### Buscar Links dos genes:

In [30]:
import time
def url_get_id(i):
    url_list= [ ]
    for id in n_genes:
        url = "https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id )
        url_list.append(url)
    return url_list
url_get_id(n_genes)

['https://www.ncbi.nlm.nih.gov/nuccore/3667',
 'https://www.ncbi.nlm.nih.gov/nuccore/3667',
 'https://www.ncbi.nlm.nih.gov/nuccore/16367',
 'https://www.ncbi.nlm.nih.gov/nuccore/25467',
 'https://www.ncbi.nlm.nih.gov/nuccore/559281',
 'https://www.ncbi.nlm.nih.gov/nuccore/3077574',
 'https://www.ncbi.nlm.nih.gov/nuccore/100512686',
 'https://www.ncbi.nlm.nih.gov/nuccore/538598',
 'https://www.ncbi.nlm.nih.gov/nuccore/459985',
 'https://www.ncbi.nlm.nih.gov/nuccore/100607873']

In [31]:
import requests

content_id = []
for url in url_get_id(n_genes):
    r = requests.get(url)
    content_id.append(r.content)
#print(content_id)

In [32]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content_id:
    soup_id = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines_id = soup_id.find_all('meta', {'name':"ncbi_uidlist"} )

    id_id = ""
    url_id = ""
    for line in lines_id:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id_id = line.attrs['content']
    if id:
        url_id = "https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&maxdownloadsize=5000000".format(id_id)

    r2 = requests.get(url_id)
    r4= r2.content.decode('utf-8')
    #print (r4, '\n')

<meta content="3667" name="ncbi_uidlist"/>
ncbi_uidlist
3667
<meta content="3667" name="ncbi_uidlist"/>
ncbi_uidlist
3667
<meta content="16367" name="ncbi_uidlist"/>
ncbi_uidlist
16367
<meta content="25467" name="ncbi_uidlist"/>
ncbi_uidlist
25467
<meta content="559281" name="ncbi_uidlist"/>
ncbi_uidlist
559281
<meta content="3077574" name="ncbi_uidlist"/>
ncbi_uidlist
3077574
<meta content="100512686" name="ncbi_uidlist"/>
ncbi_uidlist
100512686
<meta content="538598" name="ncbi_uidlist"/>
ncbi_uidlist
538598
<meta content="459985" name="ncbi_uidlist"/>
ncbi_uidlist
459985
<meta content="100607873" name="ncbi_uidlist"/>
ncbi_uidlist
100607873


### Buscar ids:

In [33]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    Ids.append(info.id)
    print(info.id)

X03907.1
AA928418.1
AAADR0035516.1
L20655.1
AAADU0045050.1


### Buscar quantidade de cds e a sua localização:

In [26]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
featcds = [ ]
position=0
pos=[]
for info in records:
    #print(info.id)
    for i in range(len(info.features)):
        if info.features[i].type == "CDS":
            featcds.append(i)
            cds= i
            a= str(cds)
            print (len(a))
            position=i
            pos.append(info.features[position].location)
            print (pos)

### Buscar Ids de proteinas

In [34]:
list_pro=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
for info in records:
    for i in info.features:
        if i.type == "CDS":
            
            pro= str(i.qualifiers["protein_id"])
            list_pro.append(pro)
div= ', '.join(list_pro)
h= div.replace("['",'')
hh= h.replace("']",'')
ID_PROT= hh.split(', ')
print (ID_PROT)

['CAA27540.1', 'AAA47844.1']


### Buscar Links de proteinas:

In [37]:

import time
ID_PROT
def url_get_id_p(a):
    url_list_p=[ ]
    for id_p in ID_PROT:
        url_id_p= "https://www.uniprot.org/uniprotkb?query={}".format(id_p)
        url_list_p.append(url_id_p)
    return url_list_p
url_ids_protein=url_get_id_p(ID_PROT)

In [36]:
def get_url(url, **kwargs):
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [45]:
# if the output is empty, it means that the protein is not reviewed 
fields="accession"
WEBSITE_API="https://rest.uniprot.org"
accession=[]

for i in ID_PROT:
    r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i,fields))
    accession.append(str(r.content))
   
for x in accession:
    print(x)

b'Entry\nP0CJ48\n'
b'Entry\nQ88364\n'


In [46]:
fields="cc_subcellular_location"
WEBSITE_API="https://rest.uniprot.org"
sub_location=[]

for i in ID_PROT:
    r=get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,i,fields))
    sub_location.append(str(r.content))
   
for x in sub_location:
    print(x)

b'Subcellular location [CC]\nSUBCELLULAR LOCATION: Plastid, chloroplast thylakoid membrane; Multi-pass membrane protein.\n'
b'Subcellular location [CC]\n\n'


In [ ]:
"accession,organism_name,protein_name,cc_subcellular_location,cc_function"

In [53]:
WEBSITE_API = "https://rest.uniprot.org"
fields = ["accession","organism_name","protein_name","cc_subcellular_location","cc_function"]

def get_field_for_id(ID_PROT, field):
    response = get_url("{}/uniprotkb/search?query={} AND (reviewed:true)&fields={}&size=1&format=tsv".format(WEBSITE_API,ID_PROT,field))
    return str(response.content)


result = {}
for field in fields:
    result[field] = [get_field_for_id(i, field) for i in ID_PROT]

for field, values in result.items():
    print(f"Results for {field}")
    for x in values:
        print(x)

Results for accession
b'Entry\nP0CJ48\n'
b'Entry\n'
Results for organism_name
b'Organism\nArabidopsis thaliana (Mouse-ear cress)\n'
b'Organism\n'
Results for protein_name
b'Protein names\nChlorophyll a-b binding protein 2, chloroplastic (Chlorophyll a-b protein 165) (CAB-165) (LHCII type I CAB-2)\n'
b'Protein names\n'
Results for cc_subcellular_location
b'Subcellular location [CC]\nSUBCELLULAR LOCATION: Plastid, chloroplast thylakoid membrane; Multi-pass membrane protein.\n'
b'Subcellular location [CC]\n'
Results for cc_function
b'Function [CC]\nFUNCTION: The light-harvesting complex (LHC) functions as a light receptor, it captures and delivers excitation energy to photosystems with which it is closely associated.\n'
b'Function [CC]\n'


In [1]:
# for a in ID_PROT:
#     with ExPASy.get_sprot_raw(a) as handle:
#             seq_record = SeqIO.read(handle, "nt")
#             print(seq_record.id)
#             print(seq_record.seq)
            

In [8]:
import requests

content_id_p = []
for url in url_get_id_p(len(ID_PROT)):
    rp = requests.get(url)
    content_id_p.append(rp.content)
#print(content_id)

In [9]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content_id_p:
    soup_id_p = BeautifulSoup(c, 'html.parser')

    # Procurar um tag meta com um determinado atributo
    lines_id_p = soup_id_p.find_all('meta', {'name':"id"} )

    id_id_p= ""
    url_id_p = ""
    for line in lines_id_p:
        print(line)
        if 'name' in line.attrs:
            print(line.attrs['name'])
        if 'content' in line.attrs:
            print(line.attrs['content'])		
            id_id_p = line.attrs['content']
    if id_id_p:
        url_id_p = "https://rest.uniprot.org/uniprotkb/search?query={} AND (reviewed:true)&fields='id'&size=1&format=tsv".format(id_id_p)

    r7 = requests.get(url_id_p)
    r8= r7.content.decode('utf-8')
    print (r8, '\n')

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant http://?

### Buscar seq:

In [ ]:
#Biopython
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
seq=[]
for info in records:
    print(info.id)
    print(info.seq[0:50],'...',info.seq[-10:])
    print()
    seq.append(f'{info.seq[0:50]}...{info.seq[-10:]}')
    

### Buscar gene description:

In [ ]:
Ids=[]
database = 'nucleotide'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="gb") 
records = list(SeqIO.parse(handle,"gb"))
handle.close()
description=[]
for info in records:
    print(info.id)
    print(info.description)
    description.append(info.description)

### Buscar artigos:

In [ ]:
Ids=[]
database = 'PubMed'
email= 'rodrigoce9@gmail.com'
idlist= n_genes
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    print("title:", info.get("TI", "-"))
    print('authors: ', info.get("AU", "-"))
    print("source:", info.get("SO", "-"))
    print("affiliation: ", info.get("AD", "-") ) # FAZER O IF/ELSE PARA O CASO DE NÃO TER AFFILIATION
    print()